In [ ]:
!pip -q install langchain openai tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.6/508.6 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install langchain

In [ ]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass()

··········


In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 4.3 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFDirectoryLoader

In [ ]:
loader = PyPDFDirectoryLoader("/content/RAKSHA")
documents = loader.load()

In [ ]:
print(documents)

[Document(page_content="Q.\nI\nhave\nscaly\npatchy\nraised\nlesions\nover\nb/l\nelbow\nand\nb/l\nknee\njoints\nwith\ndryness\nand\nitching\n.\nWhat\nis\nmy\nailment?\nWhat\nis\nthe\nsolution?\nA.\nThe\nsymptoms\nyou've\ndescribed,\nsuch\nas\nscaly\npatchy\nraised\nlesions\nover\nb/l\nelbow\nand\nb/l\nknee\njoint\nwith\ndryness\nand\nitching,\ncan\nbe\nanalyzed\nthrough\nthe\nlens\nof\nAyurveda\nto\npredict\na\npotential\nproblem\nor\ndisease.\nAyurveda\nidentifies\nimbalances\nin\nthe\nVata,\nPitta,\nand\nKapha\ndoshas\nas\nkey\nfactors\nin\nvarious\nhealth\nissues.\nVata\nDosha\n:\nScaly\nlesions\n:\nThis\nis\noften\nassociated\nwith\naggravated\nVata\ndosha.\nWhen\nVata\ndosha\nis\nimbalanced,\nit\ncan\nlead\nto\nscaly\npatchy\nraised\nlesions\n.\nWhen\nPitta\nis\naggravated,\nit\ncan\ncause\nraised\nlesions,\nredness\nof\nlesions.\nKapha\nDosha\n:\nItching\n:\nit\nis\nthe\nclassic\nkapha\ndosha\nsymptom\ni,e\nPratyatma\nlakshana\nof\nkapha\n.\nBased\non\nthese\nsymptoms\nand\ntheir\

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

10

In [ ]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)
vectordb.persist()

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["context"],
    template="""Identify possible disease and suggest a few Ayurvedic treatments.:
    The patient experiences the following symptoms: {context}
    Analyze potential imbalances in Doshas, Dhatus, and Srotas, explaining how each symptom relates to these imbalances:
    Doshas (Vata, Pitta, Kapha)
    Dhatus (Rasa, Rakta, Mamsa, Meda, Asthi, Majja, Shukra)
    Srotas (Pranavaha, Udakavaha, Annavaha, Rasavaha, Raktavaha, Mamsavaha, Medovaha, Asthivaha, Majjavaha, Sukravaha, Mutravaha, Purishavaha, Swedavaha)"""
)

In [ ]:
prompt_template.format(context = "scaly patchy raised lesions over below elbow and below knee joints with dryness and itching")

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=vector_db.as_retriever(),
                                  chain_type_kwargs = {"prompt" : prompt_template,},
                                  return_source_documents=True)

In [ ]:
import pprint

In [ ]:
pprint.pprint(qa_chain("I have scaly patchy raised lesions over below elbow and below knee joints with dryness and itching. What is the solution?predict the problem based on dhatus doshas and srotas, breaking down the symptoms mentioned"))

RateLimitError: ignored